In [ ]:
import rdflib

In [ ]:
prefixes = """
    PREFIX xl: <http://www.xbrl.org/2003/XLink/>
    PREFIX xbrll: <https://w3id.org/vocab/xbrll/>
    PREFIX xlink: <http://www.w3.org/1999/xlink/>
"""

In [ ]:
def remove_prefix(s1:str, s2:str):
    min_len = min(len(s1), len(s2))
    for i in range(1, min_len):
        if s1[:i] != s2[:i]:
            s1 = s1[i-1:]
            s2 = s2[i-1:]
            break
    return [s1, s2]

In [ ]:
schema_file = open('..\\data\\raw\\schema.ttl', encoding = 'utf-8')
rdf_schema = rdflib.Graph()
rdf_schema.parse(file=schema_file, format="turtle")

linkbase_file = open('..\\data\\raw\\linkbase.ttl', encoding = 'utf-8')
rdf_linkbase = rdflib.Graph()
rdf_linkbase.parse(file=linkbase_file, format="turtle")

In [ ]:
combinations = rdf_linkbase.query(prefixes + """
    SELECT ?loc_ref ?loc_id 
    WHERE
    { 
      ?loc_ref xl:type xbrll:locator .
      ?loc_ref <http://www.w3.org/1999/02/22-rdf-syntax-ns/id> ?loc_id .
    }
""")

In [ ]:
len(combinations)

In [ ]:
combinations = []
for triple in rdf_linkbase.triples((None, None, rdflib.term.URIRef('https://w3id.org/vocab/xbrll/locator'))):
#     found = False
    # I changed the uri here below to rdf:id (instead of xbrll:concept)
    for triple2 in rdf_linkbase.triples((triple[0],rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns/id'),None)):
        combination = [triple[0], triple2[2]]
        combinations.append(combination)
#         found = True
#     if found:
#         rdf_linkbase.remove((triple[0], None, None))        

In [ ]:
len(combinations)

In [ ]:
# replace all loc_refs by loc_ids
to_add = []
to_remove = []

# first determine which triples to add / remove
for loc_ref, loc_id in combinations:
    to_add += [(loc_id, pred, obj) for (subj, pred, obj) in rdf_linkbase.triples((loc_ref, None, None))]
    to_add += [(subj, pred, loc_id) for (subj, pred, obj) in rdf_linkbase.triples((None, None, loc_ref))]
    to_remove += [(loc_ref, pred, obj) for (subj, pred, obj) in rdf_linkbase.triples((loc_ref, None, None))]
    to_remove += [(subj, pred, loc_ref) for (subj, pred, obj) in rdf_linkbase.triples((None, None, loc_ref))]

# add and remove triples
    for triple in to_add:
    rdf_linkbase.add(triple)
for triple in to_remove:
    rdf_linkbase.remove(triple)

# delete triples with rdf:id predicate
rdf_linkbase.remove((None, 'http://www.w3.org/1999/02/22-rdf-syntax-ns/id' , None))

In [ ]:
combinations = rdf_linkbase.query(prefixes + """
    SELECT ?concept ?label
    WHERE 
    {
        ?link ?pred xbrll:concept-label .
        ?link xl:from ?concept .
        ?link xl:to ?resource .
        ?resource <http://www.w3.org/1999/02/22-rdf-syntax-ns/value> ?label .
    }
""")

In [ ]:
len(combinations)

In [ ]:
combinations = []
j = 0
for triple in rdf_linkbase.triples((None,None,rdflib.term.URIRef('https://w3id.org/vocab/xbrll/concept-label'))):
    from_key = None
    values = []
    for triple2 in rdf_linkbase.triples((triple[0],rdflib.term.URIRef('http://www.xbrl.org/2003/XLink/from'),None)):
        from_key = remove_prefix(str(triple2[0]), str(triple2[2]))[1]
    for triple2 in rdf_linkbase.triples((triple[0],rdflib.term.URIRef('http://www.xbrl.org/2003/XLink/to'),None)):
        for triple3 in rdf_linkbase.triples((triple2[2], rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns/value'), None)):
            values.append(triple3[2])
    if (from_key is not None) and (len(values) > 0):
        combinations.append([from_key, values, triple[0]])

In [ ]:
len(combinations)

In [ ]:
for subj, pred, obj in rdf_schema.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns/id'), None)):
    for combination in combinations:
        if str(obj) in combination:
            for string in combination[1]:
                rdf_schema.add((subj, rdflib.term.URIRef('https://w3id.org/vocab/xbrll/concept-label'), string))
        rdf_linkbase.remove((subj, None, None))

In [ ]:
fg = open('..\\data\\raw\\linkbase2.ttl', "w", encoding = "utf-8")
fg.write(rdf_linkbase.serialize(format='turtle').decode('utf-8'))
fg.close()

In [ ]:
fg = open('..\\data\\raw\\schema2.ttl', "w", encoding = "utf-8")
fg.write(rdf_schema.serialize(format='turtle').decode('utf-8'))
fg.close()